<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-mcp/blob/main/openai-mcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using OpenAI with a MCP Server

In [1]:
# Install required packages

!pip install --upgrade pip
!pip install fastmcp openai nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.5 MB/s  0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.1
    Uninstalling wrapt-2.0.1:
      Successfully uninstalled wrapt-2.0.1
  Attempting uninstall: referencing
    Found existing installation: referencing 0.37.0
    Uninstalling referencing-0.37.0:
      Successfully uninstalled referencing-0.37.0
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.37.0
    Uninstalling opentelemetry-api-1.37.0:
      Successfully uninstalled opentelemetry-api-1.37.0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.58b0
    Uninstalling opentelemetry-semantic-convention

In [2]:
# The MCP Server

from fastmcp import FastMCP
import nest_asyncio
import threading
import time

nest_asyncio.apply()

mcp = FastMCP(
    name="WeatherServer",
    instructions="This provides an up to date weather forecast for any location."
)

# Tool 1: Forecast
@mcp.tool("get_forecast")
def get_forecast(location: str):
    return {"forecast": f"Sunny in {location}"}

# Tool 2: Alerts
@mcp.tool(name="get_alerts")
def get_alerts(location: str):
    return {"alerts": f"No severe alerts currently for {location}"}

# Tool 3: Math
@mcp.tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers together."""
    return a * b

# Tool 4: Health Check
@mcp.tool
def health_check():
    """Returns the health status of the server."""
    return {"status": "ok"}

# Resource 1: Climate Data
@mcp.resource(uri="http://example.com", name="climate_data")
def climate_data():
    """Return static climate information."""
    return {
        "Berlin": {"avg_temp": "10°C", "rainfall": "570mm"},
        "Boise": {"avg_temp": "12°C", "rainfall": "300mm"},
        "Tokyo": {"avg_temp": "16°C", "rainfall": "1500mm"}
    }

# Prompt 1: Analysis of Numerical data
@mcp.prompt
def analyze_data(data_points: list[float]) -> str:
    """Creates a prompt asking for analysis of numerical data."""
    formatted_data = ", ".join(str(point) for point in data_points)
    return f"Please analyze these data points: {formatted_data}"

# Define the function to run the server
def run_server():
  # Use transport="streamable-http" for compatibility with notebooks/Colab
  print("🚀 Starting FastMCP server in background thread...")
  mcp.run(transport="streamable-http", host="0.0.0.0", port=8000)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to start up
time.sleep(5)
print("✅ Server should be running. Access it at http://localhost:8000/mcp")

🚀 Starting FastMCP server in background thread...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭──────────────────────────────────────────────────────────────────────────────╮                  
                 │                                                                              │                  
                 │                         ▄▀▀ ▄▀█ █▀▀ ▀█▀ █▀▄▀█ █▀▀ █▀█                        │                  
                 │                         █▀  █▀█ ▄▄█  █  █ ▀ █ █▄▄ █▀▀                        │                  
                 │                                                                              │                  
                 │                                FastMCP 2.14.1                                │                  
                 │                                                                              │                  
                 │                                                                              │                  
                 │                   🖥  Server name: WeatherServer                              │                  
                 │                                                                              │                  
                 │                   📦 Transport:   HTTP                                       │                  
                 │                   🔗 Server URL:  http://0.0.0.0:8000/mcp                    │                  
                 │                                                                              │                  
                 │                   📚 Docs:        https://gofastmcp.com                      │                  
                 │                   🚀 Hosting:     https://fastmcp.cloud                      │                  
                 │                                                                              │                  
                 ╰──────────────────────────────────────────────────────────────────────────────╯

[12/19/25 15:45:37] INFO     Starting MCP server 'WeatherServer' with transport 'streamable-http' on ]8;id=869963;file:///usr/local/lib/python3.12/dist-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=304472;file:///usr/local/lib/python3.12/dist-packages/fastmcp/server/server.py#2618\2618]8;;\
                             http://0.0.0.0:8000/mcp                                                               

INFO:     Started server process [425]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ Server should be running. Access it at http://localhost:8000/mcp


This code below sets up a test suite for the FastMCP server you started in the previous cell. It uses the fastmcp.Client to connect to the server and test its various functionalities: calling tools like get_forecast and multiply, reading resources such as climate_data, and retrieving prompts like analyze_data. The code then prints whether each test passed or failed, along with a summary of all tests. The output indicates that the client wasn't connected, which is a common issue when using fastmcp.Client outside of its recommended async with context manager.

CallToolResult(content=[TextContent(type='text', text='{"forecast":"Sunny in London"}', annotations=None, meta=None)], structured_content={'forecast': 'Sunny in London'}, meta=None, data={'forecast': 'Sunny in London'}, is_error=False)

In [13]:
import asyncio
from fastmcp import Client

# MCP_SERVER_URL is available from previous cells in the kernel state.
MCP_SERVER_URL = "http://localhost:8000/mcp"

async def run_mcp_client_tests(server_url: str):
    print(f"🚀 Starting FastMCP client tests against: {server_url}")
    print("Ensure the MCP server is running in a separate thread/process.")

    all_results = []

    async with Client(server_url) as client:
        print("\n--- Inspecting Client Object ---")
        print(f"Client type: {type(client)}")
        print(f"Client dir: {dir(client)}")
        print("--------------------------------")

        # --- Test Tools ---
        print("\n--- Running Tool Tests ---")
        tool_test_cases = [
            {'name': 'get_forecast', 'arguments': {'location': 'London'}, 'expected_result': {"forecast": "Sunny in London"}},
            {'name': 'get_alerts', 'arguments': {'location': 'London'}, 'expected_result': {"alerts": "No severe alerts currently for London"}},
            {'name': 'multiply', 'arguments': {'a': 5, 'b': 3}, 'expected_result': 15.0},
            {'name': 'health_check', 'arguments': {}, 'expected_result': {"status": "ok"}}
        ]

        for test_case in tool_test_cases:
            name = test_case['name']
            args = test_case['arguments']
            expected = test_case['expected_result']
            print(f"Testing tool: {name} with args: {args}")
            try:
                result = await client.call_tool(name, arguments=args)
                passed = (result.data == expected)
                all_results.append({
                    "type": "tool",
                    "name": name,
                    "arguments": args,
                    "passed": passed,
                    "result": result,
                    "expected": expected
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result={result}")
            except Exception as e:
                all_results.append({
                    "type": "tool",
                    "name": name,
                    "arguments": args,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

        # --- Test Resources ---
        print("\n--- Running Resource Tests ---")
        resource_test_cases = [
            {'name': 'climate_data', 'expected_result': {
                "Berlin": {"avg_temp": "10°C", "rainfall": "570mm"},
                "Boise": {"avg_temp": "12°C", "rainfall": "300mm"},
                "Tokyo": {"avg_temp": "16°C", "rainfall": "1500mm"}
            }}
        ]

        for test_case in resource_test_cases:
            name = test_case['name']
            expected = test_case['expected_result']
            print(f"Testing resource: {name}")
            try:
                result = await client.read_resource(name)
                print(expected)
                print(result)
                passed = (result == expected)
                all_results.append({
                    "type": "resource",
                    "name": name,
                    "passed": passed,
                    "result": result,
                    "expected": expected
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result={result}")
            except Exception as e:
                all_results.append({
                    "type": "resource",
                    "name": name,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

        # --- Test Prompts ---
        print("\n--- Running Prompt Tests ---")
        prompt_test_cases = [
            {'name': 'analyze_data', 'arguments': {'data_points': [10.5, 20.1, 30.0]}, 'expected_prefix': "Please analyze these data points: 10.5, 20.1, 30.0"}
        ]

        for test_case in prompt_test_cases:
            name = test_case['name']
            args = test_case['arguments']
            expected_prefix = test_case['expected_prefix']
            print(f"Testing prompt: {name} with args: {args}")
            try:
                result = await client.get_prompt(name, arguments=args)
                passed = (result == expected_prefix)
                all_results.append({
                    "type": "prompt",
                    "name": name,
                    "arguments": args,
                    "passed": passed,
                    "result": result,
                    "expected": expected_prefix
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result='{result}'")
            except Exception as e:
                all_results.append({
                    "type": "prompt",
                    "name": name,
                    "arguments": args,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

    print("\n--- Test Summary ---")
    total_tests = len(all_results)
    passed_tests = sum(1 for r in all_results if r.get('passed', False))
    failed_tests = total_tests - passed_tests

    for res in all_results:
        status = 'PASSED' if res.get('passed', False) else 'FAILED'
        print(f"[{res['type'].upper()}] {res['name']}: {status}")
        if not res.get('passed', False):
            if 'error' in res:
                print(f"  Error: {res['error']}")
            else:
                print(f"  Expected: {res.get('expected')}")
                print(f"  Got: {res.get('result')}")
    print(f"\nTotal Tests: {total_tests}, Passed: {passed_tests}, Failed: {failed_tests}")

if __name__ == "__main__":
    # Make sure to run the MCP server cell (fYO0LykYQ05Y) before running this test suite.
    asyncio.run(run_mcp_client_tests(MCP_SERVER_URL))

🚀 Starting FastMCP client tests against: http://localhost:8000/mcp
Ensure the MCP server is running in a separate thread/process.
INFO:     127.0.0.1:38052 - "POST /mcp HTTP/1.1" 200 OK

--- Inspecting Client Object ---
Client type: <class 'fastmcp.client.client.Client'>
Client dir: ['__aenter__', '__aexit__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_call_tool_as_task', '_connect', '_context_manager', '_disconnect', '_get_prompt_as_task', '_handle_task_status_notification', '_init_timeout', '_parse_call_tool_result', '_progress_handler', '_read_resource_as_task', '_reset_session_state', '_session

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
# The MCP Server test suite #2

import requests
import json
from typing import Dict, Any, List
from dataclasses import dataclass
import time

# Configuration
MCP_SERVER_URL = "http://localhost:8000/mcp"
REQUEST_ID = 1

@dataclass
class TestResult:
    name: str
    passed: bool
    message: str
    response: Any = None
    duration: float = 0.0

class MCPServerTester:
    def __init__(self, server_url: str):
        self.server_url = server_url
        self.request_id = 1
        self.results: List[TestResult] = []
        self.session_initialized = False

    def _make_request(self, method: str, params: Dict = None) -> Dict:
        """Make a JSON-RPC request to the MCP server"""
        start_time = time.time()

        payload = {
            "jsonrpc": "2.0",
            "id": self.request_id,
            "method": method,
        }

        headers={
          "Accept": "application/json, text/event-stream"
        }

        if params:
            payload["params"] = params

        try:
            response = requests.post(self.server_url, json=payload, headers=headers, timeout=10)
            duration = time.time() - start_time
            self.request_id += 1

            if response.status_code == 200:
                clean_text = response.text.removeprefix("event: message\r\ndata: ").removesuffix("\r\n\r\n")
                clean_json = json.loads(clean_text)

                # return {"success": True, "data": response.json(), "duration": duration}
                return {"success": True, "data": clean_json, "duration": duration}
            else:
                return {
                    "success": False,
                    "error": f"HTTP {response.status_code}: {response.text}",
                    "duration": duration
                }
        except Exception as e:
            duration = time.time() - start_time
            return {"success": False, "error": str(e), "duration": duration}

    def test_initialize(self) -> TestResult:
        """Test server initialization"""
        print("🔧 Testing: Server Initialization")

        result = self._make_request(
            "initialize",
            {
                "protocolVersion": "2024-11-05",
                "capabilities": {},
                "clientInfo": {
                    "name": "test-client",
                    "version": "1.0.0"
                }
            }
        )

        if result["success"]:
            data = result["data"]
            if "result" in data and "capabilities" in data["result"]:
                self.session_initialized = True
                return TestResult(
                    "initialize",
                    True,
                    "✅ Server initialized successfully",
                    data["result"],
                    result["duration"]
                )

        return TestResult(
            "initialize",
            False,
            f"❌ Initialization failed: {result.get('error', 'Unknown error')}",
            result.get("data"),
            result.get("duration", 0)
        )

    def test_list_tools(self) -> TestResult:
        """Test listing available tools"""
        print("🔧 Testing: List Tools")

        result = self._make_request("tools/list")

        if result["success"]:
            data = result["data"]
            if "result" in data and "tools" in data["result"]:
                tools = data["result"]["tools"]
                return TestResult(
                    "tools/list",
                    True,
                    f"✅ Found {len(tools)} tool(s)",
                    tools,
                    result["duration"]
                )

        return TestResult(
            "tools/list",
            False,
            f"❌ Failed to list tools: {result.get('error', 'Unknown error')}",
            result.get("data"),
            result.get("duration", 0)
        )

    def test_tool_call(self, tool_name: str, arguments: Dict = None) -> TestResult:
        """Test calling a specific tool"""
        print(f"🔧 Testing: Tool '{tool_name}'")

        params = {"name": tool_name}
        if arguments:
            params["arguments"] = arguments

        result = self._make_request("tools/call", params)

        if result["success"]:
            data = result["data"]
            if "result" in data:
                return TestResult(
                    f"tool:{tool_name}",
                    True,
                    f"✅ Tool '{tool_name}' executed successfully",
                    data["result"],
                    result["duration"]
                )
            elif "error" in data:
                return TestResult(
                    f"tool:{tool_name}",
                    False,
                    f"❌ Tool error: {data['error'].get('message', 'Unknown error')}",
                    data["error"],
                    result["duration"]
                )

        return TestResult(
            f"tool:{tool_name}",
            False,
            f"❌ Tool call failed: {result.get('error', 'Unknown error')}",
            result.get("data"),
            result.get("duration", 0)
        )

    def test_list_resources(self) -> TestResult:
        """Test listing available resources"""
        print("🔧 Testing: List Resources")

        result = self._make_request("resources/list")

        if result["success"]:
            data = result["data"]
            if "result" in data and "resources" in data["result"]:
                resources = data["result"]["resources"]
                return TestResult(
                    "resources/list",
                    True,
                    f"✅ Found {len(resources)} resource(s)",
                    resources,
                    result["duration"]
                )

        return TestResult(
            "resources/list",
            False,
            f"❌ Failed to list resources: {result.get('error', 'Unknown error')}",
            result.get("data"),
            result.get("duration", 0)
        )

    def test_read_resource(self, uri: str) -> TestResult:
        """Test reading a specific resource"""
        print(f"🔧 Testing: Resource '{uri}'")

        result = self._make_request("resources/read", {"uri": uri})

        if result["success"]:
            data = result["data"]
            if "result" in data:
                return TestResult(
                    f"resource:{uri}",
                    True,
                    f"✅ Resource '{uri}' read successfully",
                    data["result"],
                    result["duration"]
                )
            elif "error" in data:
                return TestResult(
                    f"resource:{uri}",
                    False,
                    f"❌ Resource error: {data['error'].get('message', 'Unknown error')}",
                    data["error"],
                    result["duration"]
                )

        return TestResult(
            f"resource:{uri}",
            False,
            f"❌ Resource read failed: {result.get('error', 'Unknown error')}",
            result.get("data"),
            result.get("duration", 0)
        )

    def test_list_prompts(self) -> TestResult:
        """Test listing available prompts"""
        print("🔧 Testing: List Prompts")

        result = self._make_request("prompts/list")

        if result["success"]:
            data = result["data"]
            if "result" in data and "prompts" in data["result"]:
                prompts = data["result"]["prompts"]
                return TestResult(
                    "prompts/list",
                    True,
                    f"✅ Found {len(prompts)} prompt(s)",
                    prompts,
                    result["duration"]
                )

        return TestResult(
            "prompts/list",
            False,
            f"❌ Failed to list prompts: {result.get('error', 'Unknown error')}",
            result.get("data"),
            result.get("duration", 0)
        )

    def test_get_prompt(self, name: str, arguments: Dict = None) -> TestResult:
        """Test getting a specific prompt"""
        print(f"🔧 Testing: Prompt '{name}'")

        params = {"name": name}
        if arguments:
            params["arguments"] = arguments

        result = self._make_request("prompts/get", params)

        if result["success"]:
            data = result["data"]
            if "result" in data:
                return TestResult(
                    f"prompt:{name}",
                    True,
                    f"✅ Prompt '{name}' retrieved successfully",
                    data["result"],
                    result["duration"]
                )
            elif "error" in data:
                return TestResult(
                    f"prompt:{name}",
                    False,
                    f"❌ Prompt error: {data['error'].get('message', 'Unknown error')}",
                    data["error"],
                    result["duration"]
                )

        return TestResult(
            f"prompt:{name}",
            False,
            f"❌ Prompt retrieval failed: {result.get('error', 'Unknown error')}",
            result.get("data"),
            result.get("duration", 0)
        )

    def run_all_tests(self,
                      tool_tests: List[Dict] = None,
                      resource_tests: List[str] = None,
                      prompt_tests: List[Dict] = None) -> None:
        """Run all tests and print results"""
        print("\n" + "="*80)
        print("🚀 STARTING MCP SERVER TEST SUITE")
        print("="*80 + "\n")

        # Test 1: Initialize
        result = self.test_initialize()
        self.results.append(result)
        print(f"{result.message} ({result.duration:.3f}s)\n")

        if not result.passed:
            print("⚠️  Cannot continue without successful initialization\n")
            self.print_summary()
            return

        # Test 2: List and test tools
        result = self.test_list_tools()
        self.results.append(result)
        print(f"{result.message} ({result.duration:.3f}s)")

        if result.passed and result.response:
            print("\n📋 Available Tools:")
            for tool in result.response:
                print(f"  • {tool.get('name', 'unknown')}: {tool.get('description', 'No description')}")
            print()

            # Run custom tool tests if provided
            if tool_tests:
                for test in tool_tests:
                    result = self.test_tool_call(test["name"], test.get("arguments"))
                    self.results.append(result)
                    print(f"{result.message} ({result.duration:.3f}s)\n")

        # Test 3: List and test resources
        result = self.test_list_resources()
        self.results.append(result)
        print(f"{result.message} ({result.duration:.3f}s)")

        if result.passed and result.response:
            print("\n📋 Available Resources:")
            for resource in result.response:
                print(f"  • {resource.get('uri', 'unknown')}: {resource.get('description', 'No description')}")
            print()

            # Run custom resource tests if provided
            if resource_tests:
                for uri in resource_tests:
                    result = self.test_read_resource(uri)
                    self.results.append(result)
                    print(f"{result.message} ({result.duration:.3f}s)\n")

        # Test 4: List and test prompts
        result = self.test_list_prompts()
        self.results.append(result)
        print(f"{result.message} ({result.duration:.3f}s)")

        if result.passed and result.response:
            print("\n📋 Available Prompts:")
            for prompt in result.response:
                print(f"  • {prompt.get('name', 'unknown')}: {prompt.get('description', 'No description')}")
            print()

            # Run custom prompt tests if provided
            if prompt_tests:
                for test in prompt_tests:
                    result = self.test_get_prompt(test["name"], test.get("arguments"))
                    self.results.append(result)
                    print(f"{result.message} ({result.duration:.3f}s)\n")

        self.print_summary()

    def print_summary(self) -> None:
        """Print test summary"""
        print("\n" + "="*80)
        print("📊 TEST SUMMARY")
        print("="*80 + "\n")

        passed = sum(1 for r in self.results if r.passed)
        failed = sum(1 for r in self.results if not r.passed)
        total = len(self.results)
        total_time = sum(r.duration for r in self.results)

        print(f"✅ Passed: {passed}/{total}")
        print(f"❌ Failed: {failed}/{total}")
        print(f"⏱️  Total time: {total_time:.3f}s\n")

        if failed > 0:
            print("Failed Tests:")
            for result in self.results:
                if not result.passed:
                    print(f"  • {result.name}: {result.message}")

        print("\n" + "="*80 + "\n")


# Example usage
if __name__ == "__main__":
    # Initialize tester
    tester = MCPServerTester(MCP_SERVER_URL)

    # Define your custom tests here
    # Example tool tests
    tool_tests = [
        {"name": "get_weather", "arguments": {"city": "San Francisco"}},
        {"name": "calculate", "arguments": {"expression": "2 + 2"}},
        # Add more tool tests as needed
    ]

    # Example resource tests
    resource_tests = [
        "weather://current",
        "config://settings",
        # Add more resource URIs as needed
    ]

    # Example prompt tests
    prompt_tests = [
        {"name": "weather_summary", "arguments": {"city": "New York"}},
        {"name": "help", "arguments": {}},
        # Add more prompt tests as needed
    ]

    # Run all tests
    tester.run_all_tests(
        tool_tests=tool_tests,
        resource_tests=resource_tests,
        prompt_tests=prompt_tests
    )

    # Optional: Export results to JSON
    results_json = [
        {
            "name": r.name,
            "passed": r.passed,
            "message": r.message,
            "duration": r.duration
        }
        for r in tester.results
    ]

    with open("test_results.json", "w") as f:
        json.dump(results_json, f, indent=2)

    print("💾 Results saved to test_results.json")

In [ ]:
import requests

# The /mcp endpoint is designed for POST requests with JSON-RPC payloads.
# The server requires the client to accept both application/json and text/event-stream.
response = requests.post(
    "http://localhost:8000/mcp",
    json={
        "jsonrpc": "2.0",
        "id": 1,
        "method": "initialize",
        "params": {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {
                "name": "test-client",
                "version": "1.0.0"
            }
        }
    },
    headers={
        "Accept": "application/json, text/event-stream"
    }
)

# Check the status code first
if response.status_code == 200:
    print(f"Initialization successful (Status: {response.status_code})")
    # Try to display JSON if available, otherwise display text
    try:
        display(response.json())
    except requests.exceptions.JSONDecodeError:
        display("Server did not return a JSON response for initialize, or the response was empty.")
        display(f"Response text: xxx{response.text}xxx")
else:
    display(f"Initialization failed (Status: {response.status_code})")
    display(f"Response text: {response.text}")

In [ ]:
# Set your API key
client = OpenAI(api_key="your-api-key-here")

# Now you can use the OpenAI client
# The FastMCP server is running in the background on localhost:8000
# and can be accessed by your agent if configured to do so

# Example OpenAI Agent code
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": "What's the weather like?"}
    ]
)

print(response.choices[0].message.content)